# Escribir en un pdf automáticamente, a partir del recorrido de un csv.

In [1]:
#Exportamos las librerías que vamos a utilizar
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, letter
from PyPDF2 import PdfFileWriter, PdfFileReader #Librerías requerida
from datetime import datetime, date
import io
import os

In [2]:
df = pd.read_csv('./prueba.csv')
df

,index,dni,Nombre,Apellido,Calle,Hotel,FechaEntrada
0,1,33333333,Novak,Djokovic,Amado Jorge,Fenicia,12/04/2020
1,2,44444444,Edward,Witten,Fisica Teorica,Gatorade,22/04/2020
2,3,55555555,Alan,Turing,Computacion,Redbull,05/04/2020
3,4,66666666,Cristiano,Ronaldo,Futbol,Alas,25/04/2020
4,5,77777777,Julian,Assange,Wikileaks,Terror,17/04/2020


In [3]:
def controlar_fecha(fecha_entrada):
    # Establecer formato de las fechas a introducir: dd/mm/aaaa 
    formato = "%d/%m/%Y"     
    # Introducir fecha final utilizando el formato definido
    ahora = date.today()
    fecha_ingreso = datetime.strptime(fecha_entrada, formato).date()
    salida = (ahora - fecha_ingreso).days
    return salida

In [4]:
def crear_pdf(i, cadena):
    packet = io.BytesIO()
    # Se crear un nuevo pdf utilizando ReportLab
    can = canvas.Canvas(packet, pagesize=A4)
    can.drawString(85, 625 , cadena)
    can.setFont('Times-Roman', 12)
    can.save()
    # Nos movemos al comienzo del búfer StringIO
    packet.seek(0)
    nuevo_pdf = PdfFileReader(packet)
    # Leemos el pdf base
    existe_pdf = PdfFileReader(open("./CONSTANCIA.pdf", "rb"))
    output = PdfFileWriter()
    # Se agregan los datos de la persona que será dada de alta, al pdf ya existente
    page = existe_pdf.getPage(0)
    page.mergePage(nuevo_pdf.getPage(0))
    output.addPage(page)
    # Finalmente se escribe la salida, en un archivo real
    outputStream = open("./"+ str(i+1) + "-permiso.pdf", "wb")
    output.write(outputStream)
    outputStream.close()    

In [5]:
for i in range(len(df['FechaEntrada'])):
    fecha_entrada = df['FechaEntrada'][i]
    fecha = controlar_fecha(fecha_entrada)
    if fecha > 14:
        cadena = str(df['Apellido'][i]).upper() + ' ' + str(df['Nombre'][i]).upper() + ' ' + 'DNI' + ' ' + str(df['dni'][i]).upper()
        crear_pdf(i, cadena)        